In [1]:
# installing kenlm
!sudo apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev
!wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz
!mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
!ls kenlm/build/bin

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libboost-program-options-dev is already the newest version (1.74.0.3ubuntu7).
libboost-program-options-dev set to manually installed.
libboost-system-dev is already the newest version (1.74.0.3ubuntu7).
libboost-system-dev set to manually installed.
libboost-thread-dev is already the newest version (1.74.0.3ubuntu7).
libboost-thread-dev set to manually installed.
libbz2-dev is already the newest version (1.0.8-5build1).
libbz2-dev set to manually installed.
liblzma-dev is already the newest version (5.2.5-2ubuntu1).
liblzma-dev set to manually installed.
libboost-test-dev is already the newest version (1.74.0.3ubuntu7).
libboost-test-dev set to manually installed.
cmake is already the newest version (3.22.1-1ubuntu1.22.04.1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-2ubuntu9.2).
zlib1g-dev set to manually installed.

In [2]:
# getting dataset for kenlm training
file_id ='1CyXW_wNWx9_Kjo8v8hNjFdnTBq2qmh5f'

from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download = drive.CreateFile({'id': file_id})
# Download the file to a local disc
download.GetContentFile('odia.zip')

### Making an KenLM dictionary

In [3]:
!unzip odia.zip

Archive:  odia.zip
   creating: odia/
  inflating: odia/lm_proper_nouns_ai4bharat_train.txt  
  inflating: odia/lm_train.txt       


In [4]:
#building arpa file from training data (kenLM)
!kenlm/build/bin/lmplz -o 5 <"./odia/lm_train.txt" > "5gram.arpa"

=== 1/5 Counting and sorting n-grams ===
Reading /content/odia/lm_train.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 542720 types 1574
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:18888 2:1062500672 3:1992188800 4:3187502080 5:4648440832
/content/kenlm/lm/builder/adjust_counts.cc:49 in void lm::builder::{anonymous}::StatCollector::CalculateDiscounts(const lm::builder::DiscountConfig&) threw BadDiscountException because `s.n[j] == 0'.
Could not calculate Kneser-Ney discounts for 5-grams with adjusted count 2 because we didn't observe any 5-grams with adjusted count 1; Is this small or artificial data?
Try deduplicating the input.  To override this error for e.g. a class-based model, rerun with --discount_fallback

/bin/bash: line 1:  2425 Aborted                 (core dumped) kenlm/build/bin/

In [5]:
# correcting arpa file generated from kenlm function
with open("5gram.arpa", "r") as read_file, open("5gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)


In [6]:
#!/bin/bash

# Assign the filename to a variable
input_file="./odia/lm_train.txt"



def create_models_files (input_file,output_file,lexicon_file ):

### Making the arpa files

  output_file1 =  output_file + ".arpa"
  output_file2 =  output_file + "_correct.arpa"
  output_bin_file =  output_file + ".bin"

  !kenlm/build/bin/lmplz -o 5 <"$input_file" > "$output_file1"  --discount_fallback

### adding the </s> character to the arpa file

  with open(output_file1, "r") as read_file, open(output_file2, "w") as write_file:
    has_added_eos = False
    for line in read_file:
      if not has_added_eos and "ngram 1=" in line:
        count=line.strip().split("=")[-1]
        write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
      elif not has_added_eos and "<s>" in line:
        write_file.write(line)
        write_file.write(line.replace("<s>", "</s>"))
        has_added_eos = True
      else:
        write_file.write(line)

### converting arpa file to bin file

  !kenlm/build/bin/build_binary "$output_file2" "$output_bin_file"
  print ("The file is created at :","output_bin_file")

### creating the lexicon file

  with open(input_file, 'r', encoding='utf8') as f:
    text = f.read()

# Tokenize the text into words
  words = text.split()

# Extract unique words
  unique_words = set(words)

  with open(lexicon_file, "w", encoding="utf-8") as f:
    for word in unique_words:
        # Split the word into its characters
        phonemes = " ".join(list(word)) + " |"
        # Write the word and its phonemes to the lexicon file
        # f.write(f"{word}\t{phonemes}\n")
        f.write(f"{phonemes}\n")

  return( )

In [7]:
!rm -r output_results

rm: cannot remove 'output_results': No such file or directory


In [8]:
# creating bin and lexicon file in 'output_results' folder
input_file="./odia/lm_train.txt"

!mkdir output_results
output_file =  "output_results/5gram_test"
lexicon_file =  "output_results/lexicon_test_p.txt"

create_models_files (input_file,output_file,lexicon_file )

=== 1/5 Counting and sorting n-grams ===
Reading /content/odia/lm_train.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 542720 types 1574
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:18888 2:1062500672 3:1992188800 4:3187502080 5:4648440832
Substituting fallback discounts for order 4: D1=0.5 D2=1 D3+=1.5
Statistics:
1 1574 D1=0.621083 D2=1.09639 D3+=1.69043
2 5073 D1=0.822981 D2=1.25267 D3+=1.46377
3 6361 D1=0.91229 D2=1.5109 D3+=1.29188
4 6307 D1=0.962218 D2=1.81115 D3+=1.35048
5 5709 D1=0.5 D2=1 D3+=1.5
Memory estimate for binary LM:
type     kB
probing 556 assuming -p 1.5
probing 666 assuming -r models -p 1.5
trie    254 without quantization
trie    143 assuming -q 8 -b 8 quantization 
trie    241 assuming -a 22 array pointer compression
trie    130 assuming -a 22 -q 8 -b 8 array pointer com

()